# Import ELK Snapshots onto a Fabric slice

### Prerequisite
- A running fabric slice (For example: [Create single node backup Slice](./create_single_node_backup.ipynb))
- An ELK snapshot tar file to import

#### General Imports

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
except Exception as e:
    print(f"Exception: {e}")

#### Get your slice information
*Set slice_name and node_name below before running*

In [ ]:
slice_name='backup-testing'
node_name='data-backup'


slice = fablib.get_slice(name=slice_name)
slice.show();
node = slice.get_node(name=node_name)
node.show();

#### Import ELK import/export service tools

In [ ]:
from mflib.data_transfer import elk_service
elk_tools = elk_import_service(slice_name=slice_name, node_name=node_name)

# Instructions to start ELK and import your snapshot data

#### Ensure Docker and Docker-Compose are installed on the node

In [ ]:
elk_tools.install_docker()

#### Set up NAT64 Server
We need to do this step so that we can clone the Github repo with our docker app.

In [ ]:
elk_tools.setup_nat64(node_name)

### Create docker container and import your data

#### Clone Docker App Repository
Clones https://github.com/jackhancockuky/elk-docker-container.git to your docker container.
This repository contains a docker-compose app which will allow you to automatically set up Elastic Search and Kibana on your node.

In [ ]:
elk_tools.clone_repository()

#### Upload a snapshot file into ELK

In [ ]:
# Enter your elk snapshot tar file name (must be in data_transfer_service/snapshots directory)
snapshot_tar_name = "elk_backup_2.tar"

# 1. Uploads TAR file to your node
# 2. Untars data and places inside ELK's shared docker volume
elk_tools.import_snapshot(snapshot_tar_name)

#### Start the docker compose app

In [ ]:
elk_tools.start_docker()

#### Register the repository

In [ ]:
# Choose a name for the repository your snapshot will be imported into or keep as default
repository_name = "imported_repository"

elk_tools.register_repository(repository_name)

#### View repository after registration

In [ ]:
elk_tools.view_repository("imported_repository")

#### Delete all indices and restore snapshot (replace snapshot name and change index list if you want to customize)

In [ ]:
repository_name = "imported_repository"
snapshot_name = "snapshot_test"

elk_tools.restore_snapshot(repository_name, snapshot_name)

### Your data is now imported on the node.

----
### View the index list

In [ ]:
elk_tools.view_indices()

----
### View data from Kibana

#### On your local terminal create a tunnel with:

'ssh -L 10030:localhost:5601 -F ssh_config -i slice_key ubuntu@{node_ip}'

#### Then go to http://localhost:10030
----

### To reuse slice with new data (if you are done then just delete the slice)
#### Take down container and remove data

In [ ]:
#elk_tools.stop_docker()
#elk_tools.remove_data()

#### Rerun steps starting from *Create Docker container and import your data*